# News headline topic analysis with Latent Dirichlet allocation (LDA)
The purpose of this analysis is to find dominant topics across news headlines (25 per day, over 1 year). These topics will later be correlated with daily stock market loss/gain information to understand how certain topics influence the stock market.

## What is LDA?
Latent Dirichlet allocation is a way to discover higher-level topics out of individual words present in any set of document (in our case, news headlines). You can use LDA to get a sense of the overall themes in a set of documents. 

LDA is an unsupervised machine learning model that works by analyzing two things: a distribution of topics in a document, and a distribution of words in a topic. The "topics" it finds aren't specific words (e.g., "This headline is about 'war'") but instead conceptually similar groups of words that together make up a theme (e.g., "This headline is similar to the words 'war', 'crisis', 'iran'...").

Once you've created your LDA model, you can feed in a document and the model will score the overall relevancy of your document against the main _x_ topics found in your overall corpus. In other words, it will tell you which of the main topics found in the overall corpus are also found in your document, and to what extent.

## Why LDA?
LDA is a fairly popular topic modelling choice among NLP professionals, and relatively straightforward to implement. Using LDA, we were able to find 10 dominant themes in eight years' worth of news headlines within about an hour—a task that would take a human days to analyze.

## How does text pre-processing affect the accuracy of LDA?
We tested two pre-processing approaches: Bag of Words and TF-IDF, each of which count how often words appear in a document. The TF-IDF model tries to understand which words are more important than others in a document, and scores them accordingly. Presumably, TF-IDF preprocessing will help train a more accurate LDA model.

## How does LDA perform compared to other topic models?
More here later.

## Credit
Parts of this work borrow from excellent Susan Li's ["Topic Modeling and LDA in Python"](https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24) tutorial.

# Load the data from CSV, dependencies

In [1]:
# import dependencies
import pandas as pd

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import numpy as np
np.random.seed(22)

[nltk_data] Downloading package wordnet to /Users/stacy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Import the data
data = pd.read_csv("../Data/RedditNews.csv")

In [3]:
data.head()

# 73,608 records from 2016-2018

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host
2,2016-07-01,"The president of France says if Brexit won, so..."
3,2016-07-01,British Man Who Must Give Police 24 Hours' Not...
4,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...


In [4]:
# Getting just the headlines for our corpus
headlines = data[['News']]
headlines.head()

,News
0,A 117-year-old woman in Mexico City finally re...
1,IMF chief backs Athens as permanent Olympic host
2,"The president of France says if Brexit won, so..."
3,British Man Who Must Give Police 24 Hours' Not...
4,100+ Nobel laureates urge Greenpeace to stop o...


# Data pre-processing
## Lemmitize
In order to properly identify topics, we need to standardize the words in some way, removing extraneous information like tenses or plurals (e.g. "running", "runs" -> "run"). Lemmatization is the best way to do that, although it's not always the fastest way. In some cases, stemming will also work, althought it is not as precise.

Since we have a relatively small corpus to study, we will use lemmatization in order to be as precise as possible when cleaning up the words. To do this, we will feed in a `pos` parameter that tells our lemmatizing function to perform stemming with context.

Natural Language Toolkit's `stem` package includes a handy function called `WordNetLemmatizer` that makes lemmatizing our words simple. 

In [5]:
# Lemmatize the words keeping the context (stemming is "dumb" so we won't)
# However if we have a much larger corpus, we might consider stemming (as it is faster)
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v') # pos='v' means it peforms stemming with context

## Remove stopwords and words shorter than 3 chars
We also need to remove common words that don't add value ("a", "the", etc). These are called _stopwords_, and most NLP libraries have built-in methods for dealing with them. In `gensim`, the stopwords list can be called using `gensim.parsing.preprocessing.STOPWORDS`.

In [6]:
# Remove stopwords and words shorter than 3 characters, then lemmatize
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize(token))
    return result

## Check outputs
Let's compare the regular version of a sample headlind with the tokenized and lemmatized version. You'll note they are slightly different, but retain the same context.

In [7]:
sample = headlines['News'][2]

print('original document: ')
words = []
for word in sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(sample))

original document: 
['The', 'president', 'of', 'France', 'says', 'if', 'Brexit', 'won,', 'so', 'can', 'Donald', 'Trump']


 tokenized and lemmatized document: 
['president', 'france', 'say', 'brexit', 'donald', 'trump']


## Preprocess the headlines and save the results
Now we feed all the headlines into the preprocessing function, which lemmatizes and removes stopwords. 

In [8]:
cleaned_headlines = headlines['News'].map(preprocess)
cleaned_headlines[:5] # Check the results

0    [year, woman, mexico, city, finally, receive, ...
1      [chief, back, athens, permanent, olympic, host]
2      [president, france, say, brexit, donald, trump]
3    [british, police, hours, notice, threaten, hun...
4    [nobel, laureates, urge, greenpeace, stop, opp...
Name: News, dtype: object

## Count the word occurences using Bag of Words
We can count the words using different approaches. Bag of Words will simply count all remaining words left in our corpus.

In [9]:
# First we need to convert the words to numeric indices. We can do that with a gensim Dictionary.

# corpora.Dictionary implements the concept of a Dictionary – a mapping between words and their integer ids.
# https://radimrehurek.com/gensim/corpora/dictionary.html

dictionary = gensim.corpora.Dictionary(cleaned_headlines)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 alvarez
1 bear
2 birth
3 certificate
4 city
5 die
6 finally
7 hours
8 later
9 lira
10 mexico


In [10]:
'''
Filter out irrelevant words:
Keep tokens that appear in at least 15 documents
Keep only the 100,000 most frequent tokens
'''

dictionary.filter_extremes(no_below=15, keep_n=100000)

In [11]:
'''
Apply `doc2bow` to each headline, which createst a Bag of Words dictionary that reports how many words 
are in the headline and how many times each word appears.

Save this to an array, then check the example headline from above.
'''

bow_corpus = [dictionary.doc2bow(headline) for headline in cleaned_headlines]
print(bow_corpus[2])

bow_doc_2 = bow_corpus[2]

for i in range(len(bow_doc_2)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_2[i][0], 
                                            dictionary[bow_doc_2[i][0]], 
                                            bow_doc_2[i][1]))

[(21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]
Word 21 ("brexit") appears 1 time.
Word 22 ("donald") appears 1 time.
Word 23 ("france") appears 1 time.
Word 24 ("president") appears 1 time.
Word 25 ("say") appears 1 time.
Word 26 ("trump") appears 1 time.


## Count the word occurences using TF-IDF
We'll also test whether TF-IDF works better than Bag of Words for creating a corpus that, when fed into our LDA model, will produce coherent and accurate topics. 

In `gensim`, TF-IDF builds on Bag of Words, taking in processed BoW corpus data and returning a number that reflects how important a word is to each headline in the overall corpus. Conversely, the scikit-learn implementation of TF-IDF handles this all in one step.

In [12]:
from gensim import corpora, models

# Create the model
tfidf = models.TfidfModel(bow_corpus)

# Feed in the BoW corpus to the TF-IDF model
corpus_tfidf = tfidf[bow_corpus]

# Print the same headline from above
corpus_tfidf[2]

[(21, 0.5271930944921088),
 (22, 0.5289685708527566),
 (23, 0.3076906708092066),
 (24, 0.25211992539378436),
 (25, 0.18133987533447585),
 (26, 0.5011362450471095)]

## Running LDA using Bag of Words
We'll first run `LdaMulticore` using our `bow_corpus` and limiting our number of topics to 10. We're also going to print out the resulting topics and how the dominant words score in each topic. The parameter `id2word` helps us pass the numeric indices in our dictionary back into human-readable words.

In [13]:
# We have to play around with how many passes to take over the corpus in order to make the topics coherent
lda_model_bow = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2)

In [14]:
# Print each topic and how the words in those topics score (their relative weight)
for idx, topic in lda_model_bow.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.026*"israel" + 0.025*"china" + 0.014*"iran" + 0.013*"state" + 0.013*"say" + 0.012*"attack" + 0.010*"world" + 0.008*"right" + 0.008*"human" + 0.007*"nuclear"
Topic: 1 
Words: 0.012*"bank" + 0.010*"government" + 0.009*"court" + 0.008*"million" + 0.008*"police" + 0.007*"chinese" + 0.007*"protest" + 0.007*"china" + 0.006*"say" + 0.006*"billion"
Topic: 2 
Words: 0.031*"kill" + 0.018*"attack" + 0.014*"force" + 0.012*"bomb" + 0.010*"soldier" + 0.009*"army" + 0.008*"syria" + 0.008*"state" + 0.008*"taliban" + 0.008*"rebel"
Topic: 3 
Words: 0.023*"north" + 0.022*"korea" + 0.021*"kill" + 0.018*"south" + 0.013*"pakistan" + 0.010*"strike" + 0.009*"say" + 0.008*"death" + 0.008*"year" + 0.008*"people"
Topic: 4 
Words: 0.031*"israel" + 0.026*"gaza" + 0.014*"israeli" + 0.013*"palestinian" + 0.010*"ship" + 0.009*"hamas" + 0.008*"right" + 0.008*"west" + 0.008*"say" + 0.007*"report"
Topic: 5 
Words: 0.011*"world" + 0.010*"india" + 0.007*"water" + 0.007*"power" + 0.007*"years" + 0.007*"p

## Running LDA using TF-IDF
We'll run the same LDA model on the corpus processed using TF-IDF, to see if the topics generated here are any more meaningful. We'll use the same other parameters in our `LdaMulticore` model, for consistency.

In [15]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.008*"nuclear" + 0.007*"iran" + 0.005*"world" + 0.005*"israel" + 0.005*"earthquake" + 0.005*"sanction" + 0.004*"china" + 0.004*"say" + 0.004*"arm" + 0.004*"bank"
Topic: 1 Word: 0.011*"georgia" + 0.005*"russia" + 0.004*"world" + 0.004*"china" + 0.004*"marriage" + 0.004*"price" + 0.004*"year" + 0.003*"death" + 0.003*"food" + 0.003*"israel"
Topic: 2 Word: 0.008*"mugabe" + 0.007*"zimbabwe" + 0.006*"chavez" + 0.005*"russia" + 0.005*"venezuela" + 0.004*"picture" + 0.004*"china" + 0.004*"israel" + 0.004*"world" + 0.004*"mumbai"
Topic: 3 Word: 0.007*"iran" + 0.006*"kill" + 0.005*"arrest" + 0.005*"israel" + 0.005*"attack" + 0.005*"saudi" + 0.005*"pakistan" + 0.004*"strike" + 0.004*"somalia" + 0.004*"police"
Topic: 4 Word: 0.011*"korea" + 0.009*"north" + 0.006*"pirate" + 0.006*"south" + 0.004*"party" + 0.004*"japan" + 0.004*"world" + 0.004*"berlusconi" + 0.003*"china" + 0.003*"year"
Topic: 5 Word: 0.008*"police" + 0.005*"video" + 0.005*"israeli" + 0.004*"wikileaks" + 0.004*"dead"

## Compare the performance of the two preprocessing steps in the LDA model

In [16]:
# our test headline from above
cleaned_headlines[2]

['president', 'france', 'say', 'brexit', 'donald', 'trump']

### Bag of Words LDA model scores

In [17]:
for index, score in sorted(lda_model_bow[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_bow.print_topic(index, 10)))


Score: 0.871357798576355	 
Topic: 0.010*"say" + 0.009*"saudi" + 0.008*"election" + 0.008*"vote" + 0.008*"government" + 0.007*"country" + 0.007*"party" + 0.006*"leave" + 0.006*"drug" + 0.006*"european"

Score: 0.014297503978013992	 
Topic: 0.011*"government" + 0.009*"internet" + 0.008*"israeli" + 0.007*"say" + 0.006*"muslim" + 0.006*"president" + 0.006*"people" + 0.006*"protest" + 0.005*"right" + 0.005*"france"

Score: 0.014294767752289772	 
Topic: 0.022*"police" + 0.012*"russian" + 0.010*"president" + 0.009*"russia" + 0.009*"year" + 0.008*"woman" + 0.007*"putin" + 0.007*"georgia" + 0.007*"shoot" + 0.006*"say"

Score: 0.014294173568487167	 
Topic: 0.024*"iran" + 0.015*"russia" + 0.014*"world" + 0.014*"nuclear" + 0.011*"iraq" + 0.008*"china" + 0.007*"say" + 0.006*"obama" + 0.006*"missile" + 0.005*"military"

Score: 0.01429341733455658	 
Topic: 0.031*"kill" + 0.018*"attack" + 0.014*"force" + 0.012*"bomb" + 0.010*"soldier" + 0.009*"army" + 0.008*"syria" + 0.008*"state" + 0.008*"taliban" +

### TF-IDF LDA model scores

In [18]:
for index, score in sorted(lda_model_tfidf[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index)))


Score: 0.5387398600578308	 
Topic: 0.008*"police" + 0.005*"video" + 0.005*"israeli" + 0.004*"wikileaks" + 0.004*"dead" + 0.004*"shoot" + 0.004*"riot" + 0.004*"ossetia" + 0.004*"attack" + 0.003*"iran"

Score: 0.34688863158226013	 
Topic: 0.008*"nuclear" + 0.007*"iran" + 0.005*"world" + 0.005*"israel" + 0.005*"earthquake" + 0.005*"sanction" + 0.004*"china" + 0.004*"say" + 0.004*"arm" + 0.004*"bank"

Score: 0.014297934249043465	 
Topic: 0.005*"drug" + 0.005*"mexico" + 0.004*"kill" + 0.004*"world" + 0.004*"mexican" + 0.004*"people" + 0.004*"china" + 0.004*"say" + 0.004*"police" + 0.004*"dead"

Score: 0.014296957291662693	 
Topic: 0.006*"israel" + 0.005*"right" + 0.004*"internet" + 0.004*"israeli" + 0.004*"palestinians" + 0.003*"say" + 0.003*"bank" + 0.003*"world" + 0.003*"jewish" + 0.003*"china"

Score: 0.014296481385827065	 
Topic: 0.008*"mugabe" + 0.007*"zimbabwe" + 0.006*"chavez" + 0.005*"russia" + 0.005*"venezuela" + 0.004*"picture" + 0.004*"china" + 0.004*"israel" + 0.004*"world" + 0

Both approaches have relatively high accuracy with classification (> .80), and mostly make sense when reviewed by a human. What if we predicted topics for a headline that does not appear in our training set?

### Testing models on a new headline
When using an unseen headline, the Bag of Words-trained LDA model appears to classify the headline's topic more accurately.

* Bag of Words top topic: `Topic: 0.030*"iran" + 0.022*"north" + 0.022*"korea" + 0.018*"south" + 0.016*"russia"`
* TF-IDF top topic: `Topic: 0.014*"gaza" + 0.011*"israel" + 0.007*"israeli" + 0.006*"drug" + 0.006*"mugabe"`

In [19]:
unseen_document = 'Trump finally broke with Fauci'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

print("BoW approach\n")
for index, score in sorted(lda_model_bow[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_bow.print_topic(index, 5)))
    
print("\nTF-IDF approach")
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

BoW approach

Score: 0.4922909736633301	 Topic: 0.024*"iran" + 0.015*"russia" + 0.014*"world" + 0.014*"nuclear" + 0.011*"iraq"
Score: 0.3075962960720062	 Topic: 0.010*"say" + 0.009*"saudi" + 0.008*"election" + 0.008*"vote" + 0.008*"government"
Score: 0.025018606334924698	 Topic: 0.012*"bank" + 0.010*"government" + 0.009*"court" + 0.008*"million" + 0.008*"police"
Score: 0.02501637674868107	 Topic: 0.011*"government" + 0.009*"internet" + 0.008*"israeli" + 0.007*"say" + 0.006*"muslim"
Score: 0.025015847757458687	 Topic: 0.022*"police" + 0.012*"russian" + 0.010*"president" + 0.009*"russia" + 0.009*"year"
Score: 0.02501426450908184	 Topic: 0.026*"israel" + 0.025*"china" + 0.014*"iran" + 0.013*"state" + 0.013*"say"
Score: 0.02501264214515686	 Topic: 0.031*"israel" + 0.026*"gaza" + 0.014*"israeli" + 0.013*"palestinian" + 0.010*"ship"
Score: 0.025012027472257614	 Topic: 0.023*"north" + 0.022*"korea" + 0.021*"kill" + 0.018*"south" + 0.013*"pakistan"
Score: 0.025011582300066948	 Topic: 0.031*"ki

## Export the model
We'll use the LDA-Bag of words model, which appears to be more accurate (based on our test documents).

In [20]:
import joblib
joblib.dump(lda_model_bow, "lda_bow.gz")

['lda_bow.gz']

## Append topics and scores to cleaned dataset

In [21]:
# This function returns the best-scoring topic and score assocated with a headline
def topic_getter(headline):
    if headline == np.nan:
        return np.nan
    else:
        processed_headline = dictionary.doc2bow(preprocess(headline))
        topic_scores = sorted(lda_model_bow[processed_headline], key=lambda tup: -1*tup[1])
        topic = int(topic_scores[0][0])
        return topic
#         topic = topic_scores[0][0]
#         score = topic_scores[0][1]
#         return [topic, score]

topic_getter(unseen_document)

1

In [22]:
# Load dataset with combined data
all_data = pd.read_csv('../Data/Combined_News_DJIA.csv')
all_data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [23]:
all_data.dropna(how='any', inplace=True)

In [24]:
# Loop through the columns containing headlines and apply `topic_getter` to each headline, to retrieve topic
# Assign that data to a new column

for x in range(1,26): # for each column
    col = "Top" + str(x)
    new_col = col + "_topic"
    
    # for each row in column
    # get the headline, feed it into the lda_model_bow to get the topic and score
    all_data[new_col] = all_data[col].apply(topic_getter)

In [25]:
# Check the output
all_data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16_topic,Top17_topic,Top18_topic,Top19_topic,Top20_topic,Top21_topic,Top22_topic,Top23_topic,Top24_topic,Top25_topic
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,6,2,0,7,2,7,6,2,7,6
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,0,6,6,0,0,4,6,6,7,5
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,6,6,0,6,1,2,6,9,9,5
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,3,7,1,4,0,0,7,4,4,6
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,0,0,4,4,6,9,2,4,2,2


In [26]:
# Save updated data
all_data.to_csv("../Data/Combined_News_DJIA_topics.csv")

# Find one dominant topic per day

In [27]:
single_topic = all_data
del all_data # Removing now that unneeded to improve performance

In [28]:
# Drop unnecessary columns from prev dataframe
cols = []
for x in range(1,26):
    cols.append("Top" + str(x) + "_topic")  

single_topic.drop(axis=1, labels=cols, inplace=True)
single_topic.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [78]:
# Combine, clean single_topic headlines 
single_topic.loc[:,'combined_headlines'] = single_topic.loc[:,'Top1'] + " " + single_topic.loc[:,'Top2'] \
                                        + " " + single_topic.loc[:,'Top3'] + " " + single_topic.loc[:,'Top4'] \
                                        + " " + single_topic.loc[:,'Top5'] + " " + single_topic.loc[:,'Top6'] \
                                        + " " + single_topic.loc[:,'Top7'] + " " + single_topic.loc[:,'Top8'] \
                                        + " " + single_topic.loc[:,'Top9'] + " " + single_topic.loc[:,'Top10'] \
                                        + " " + single_topic.loc[:,'Top11'] + " " + single_topic.loc[:,'Top12'] \
                                        + " " + single_topic.loc[:,'Top13'] + " " + single_topic.loc[:,'Top14'] \
                                        + " " + single_topic.loc[:,'Top15'] + " " + single_topic.loc[:,'Top16'] \
                                        + " " + single_topic.loc[:,'Top17'] + " " + single_topic.loc[:,'Top18'] \
                                        + " " + single_topic.loc[:,'Top19'] + " " + single_topic.loc[:,'Top20'] \
                                        + " " + single_topic.loc[:,'Top21'] + " " + single_topic.loc[:,'Top22'] \
                                        + " " + single_topic.loc[:,'Top23'] + " " + single_topic.loc[:,'Top24'] \
                                        + " " + single_topic.loc[:,'Top25']

single_topic.loc[:,'combined_headlines'] = single_topic.loc[:,'combined_headlines'].str.replace('b[\'\"]', ' ')
single_topic.dropna(how='any', inplace=True)

cleaned_all_headlines = single_topic['combined_headlines'].map(preprocess)

single_topic.iloc[905,:]

Date                                                         2012-03-16
Label                                                                 0
Top1                  Chinese official proposes death penalty as det...
Top2                  New Chevron Oil Leak Off Coast of Brazil - Jus...
Top3                    Shell admits to at least 207 oil spills in 2011
Top4                  A Jewish man wins his fight against a German m...
Top5                  UK would not help Israel attack Iran, Prime Mi...
Top6                  "Afghan President Hamid Karzai lashed out at t...
Top7                  Putin's Fabled Tiger Encounter was PR Stunt, S...
Top8                  Moroccans call for end to rape-marriage laws -...
Top9                  Friends no more? Egypts MPs declare Israel No....
Top10                 In emails, the nuclear physicist at one of Eur...
Top11                 Syrian armed forces desertion said to surge to...
Top12                 A Moroccan government minister on Thursday

In [79]:
# Count word occurences using Bag of Words on new cleaned_headlines
dictionary = gensim.corpora.Dictionary(cleaned_all_headlines)

In [83]:
'''
Filter out irrelevant words:
Keep tokens that appear in at least 15 documents
Keep only the 100,000 most frequent tokens
'''

dictionary.filter_extremes(no_below=15, keep_n=100000)

In [84]:
'''
Apply `doc2bow` to each headline, which createst a Bag of Words dictionary that reports how many words 
are in the headline and how many times each word appears.

Save this to an array, then check the example headline from above.
'''

bow_corpus = [dictionary.doc2bow(headline) for headline in cleaned_all_headlines]
print(bow_corpus[2])

bow_doc_2 = bow_corpus[2]

for i in range(len(bow_doc_2)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_2[i][0], 
                                            dictionary[bow_doc_2[i][0]], 
                                            bow_doc_2[i][1]))

[(12, 1), (13, 1), (15, 1), (21, 1), (30, 1), (33, 10), (34, 4), (42, 1), (44, 1), (45, 1), (47, 1), (58, 1), (59, 3), (62, 2), (63, 1), (65, 1), (66, 1), (67, 2), (68, 3), (78, 1), (83, 5), (84, 2), (94, 4), (95, 1), (103, 2), (106, 1), (111, 1), (121, 1), (125, 1), (136, 1), (139, 2), (199, 1), (200, 1), (201, 1), (202, 1), (203, 1), (204, 1), (205, 1), (206, 1), (207, 1), (208, 1), (209, 1), (210, 1), (211, 1), (212, 1), (213, 1), (214, 1), (215, 1), (216, 1), (217, 1), (218, 1), (219, 1), (220, 1), (221, 1), (222, 1), (223, 1), (224, 1), (225, 1), (226, 1), (227, 1), (228, 1), (229, 2), (230, 2), (231, 1), (232, 1), (233, 1), (234, 1), (235, 1), (236, 1), (237, 1), (238, 1), (239, 2), (240, 1), (241, 1), (242, 1), (243, 1), (244, 1), (245, 1), (246, 1), (247, 1), (248, 1), (249, 1), (250, 1), (251, 1), (252, 1), (253, 1), (254, 1), (255, 2), (256, 1), (257, 2), (258, 1), (259, 1), (260, 1), (261, 1), (262, 1), (263, 1), (264, 1), (265, 1), (266, 1), (267, 1), (268, 1), (269, 1), (2

In [85]:
# Run new BoW data through LDA model

'''
NOTE: Run this on a machine with lots of processing power, or as a Google Colab notebook. Otherwise it might
take a really long time to run! TODO: Refactor this notebook to optimize performance so it can be run locally.
'''

lda_model_bow = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2)

Process ForkPoolWorker-16:
Process ForkPoolWorker-17:
Traceback (most recent call last):
  File "/Users/stacy/opt/anaconda3/envs/PythonData/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/stacy/opt/anaconda3/envs/PythonData/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/stacy/opt/anaconda3/envs/PythonData/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/stacy/opt/anaconda3/envs/PythonData/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)


KeyboardInterrupt: 

  File "/Users/stacy/opt/anaconda3/envs/PythonData/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/stacy/opt/anaconda3/envs/PythonData/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/Users/stacy/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 334, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
  File "/Users/stacy/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 334, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
  File "/Users/stacy/opt/anaconda3/envs/PythonData/lib/python3.6/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/Users/stacy/opt/anaconda3/envs/PythonData/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/Users/stacy/opt/anaconda3/envs/PythonData/lib/py

In [ ]:
# Print each topic and how the words in those topics score (their relative weight)
for idx, topic in lda_model_bow.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
# Test new model on unseen dataset
unseen_document = 'Trump finally broke with Fauci'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

print("BoW approach\n")
for index, score in sorted(lda_model_bow[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_bow.print_topic(index, 10)))
    
# This has a higher accuracy score than the per-headline topic analysis above

In [36]:
# This function returns the best-scoring topic and score assocated with a headline
def topic_getter(headline):
    if headline == np.nan:
        return np.nan
    else:
        processed_headline = dictionary.doc2bow(preprocess(headline))
        topic_scores = sorted(lda_model_bow[processed_headline], key=lambda tup: -1*tup[1])
        topic = int(topic_scores[0][0])
        return topic
    
def word_getter(headline):
    if headline == np.nan:
        return np.nan
    else:
        processed_headline = dictionary.doc2bow(preprocess(headline))
        topic_scores = sorted(lda_model_bow[processed_headline], key=lambda tup: -1*tup[1])
        topic = int(topic_scores[0][0])
        return lda_model_bow.print_topic(topic, 10)    

# Test to confirm it still returns the topic for a particular headline
topic_getter(unseen_document)
word_getter(unseen_document)

'0.023*"world" + 0.013*"president" + 0.010*"china" + 0.008*"say" + 0.007*"country" + 0.006*"people" + 0.006*"government" + 0.006*"million" + 0.006*"australia" + 0.005*"mugabe"'

In [37]:
single_topic['daily_topic'] = single_topic['combined_headlines'].apply(topic_getter)
single_topic['daily_words'] = single_topic['combined_headlines'].apply(word_getter)

In [38]:
single_topic.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top19,Top20,Top21,Top22,Top23,Top24,Top25,combined_headlines,daily_topic,daily_words
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",Georgia 'downs two Russian warplanes' as coun...,4,"0.024*""kill"" + 0.019*""iraq"" + 0.013*""force"" + ..."
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",Why wont America and Nato help us? If they wo...,4,"0.024*""kill"" + 0.019*""iraq"" + 0.013*""force"" + ..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,"b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",Remember that adorable 9-year-old who sang at...,4,"0.024*""kill"" + 0.019*""iraq"" + 0.013*""force"" + ..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,"b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,U.S. refuses Israel weapons to attack Iran: ...,4,"0.024*""kill"" + 0.019*""iraq"" + 0.013*""force"" + ..."
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,"b'""Non-media"" ph

In [39]:
# Save updated data
single_topic.to_csv("../Data/Combined_News_DJIA_single_topic.csv", index=False)